In [1]:
%load_ext autoreload
%autoreload 2
import sys
import torch
from torch import nn
sys.path.append('..')
# sys.path.append('/system/user/beck/pwbeck/projects/regularization/ml_utilities')
from pathlib import Path
from typing import Union
from ml_utilities.torch_models.base_model import BaseModel
from ml_utilities.torch_models.fc import FC
from ml_utilities.torch_models import get_model_class
from omegaconf import OmegaConf

from erank.utils import load_directions_matrix_from_task_sweep
import matplotlib.pyplot as plt
from erank.output_loader.job_output import JobResult, SweepResult
from erank.utils import get_best_model_idx

gpu_id = 0

## JobResult class

In [2]:
job_dir = '/system/user/beck/pwbeck/projects/regularization/erank/outputs/mnist-11.7.0-lenet_rotatedtasks--221015_124528/outputs/mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-0.0-seed-0--221015_195056'

In [3]:
jobr = JobResult(job_dir = job_dir)

In [4]:
str(jobr)

'/system/user/beck/pwbeck/projects/regularization/erank/outputs/mnist-11.7.0-lenet_rotatedtasks--221015_124528/outputs/mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-0.0-seed-0--221015_195056'

In [5]:
jobr.is_successful_job()

True

In [6]:
jobr.get_config()

{'run_config': {'exec_type': 'parallel', 'hostname': 'node01', 'gpu_ids': [0, 1, 2], 'runs_per_gpu': 6, 'wandb': {'init': {'tags': ['${config.experiment_data.experiment_tag}_exps', 'run_handler'], 'notes': None, 'group': '${config.experiment_data.experiment_tag}', 'job_type': 'run_handler'}}}, 'seeds': [0], 'sweep': {'type': 'grid', 'axes': [{'parameter': 'trainer.init_model_step', 'vals': [0, 5, 15, 50, 100]}, {'parameter': 'data.dataset_kwargs.rotation_angle', 'vals': 'linspace(0,180,360,endpoint=True)'}]}, 'start_num': 0, 'config': {'experiment_data': {'entity': 'jkuiml-fsl', 'project_name': 'sparsity', 'experiment_tag': '11.7', 'experiment_type': 'startnum_${start_num}', 'experiment_name': 'mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-0.0-seed-0', 'experiment_dir': None, 'experiment_notes': 'Hyperparameter search.', 'job_name': None, 'seed': 0, 'hostname': 'node01', 'gpu_id': 1}, 'model': {'name': 'fc', 'model_kwargs': {'input_size': 784, 'hidden_sizes': [300, 

In [7]:
jobr._job_dir.name, jobr.directory

('mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-0.0-seed-0--221015_195056',
 PosixPath('/system/user/beck/pwbeck/projects/regularization/erank/outputs/mnist-11.7.0-lenet_rotatedtasks--221015_124528/outputs/mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-0.0-seed-0--221015_195056'))

In [8]:
jobr.get_summary()

,best_step,best_val_score,trainer.init_model_step,data.dataset_kwargs.rotation_angle,seed
mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-0.0-seed-0--221015_195056,1550,0.97675,0,0.0,0


In [31]:
df = jobr.get_data_log('val')
df

,epoch,train_step,loss_CrossEntropyLoss,Accuracy,time_last_val_epoch_in_s,log_step
0,0,0,2.307779,0.100833,0.000000,0
1,1,50,0.364136,0.895417,6.059760,51
2,1,100,0.296845,0.913250,4.795486,102
3,1,150,0.248924,0.925750,3.255804,153
4,1,200,0.215796,0.936167,2.351608,204
5,1,250,0.183639,0.946333,3.766966,255
6,1,300,0.178382,0.945417,3.088484,306
7,1,350,0.160515,0.951083,3.795387,357
8,2,400,0.147858,0.957500,5.072853,409
9,2,450,0.134934,0.959833,5.386367,460


In [33]:
row_sel = ('train_step', 500)
col_sel = ['Accuracy']
d = df[df[row_sel[0]]==row_sel[1]][['loss_CrossEntropyLoss', 'Accuracy']].transpose().to_dict()

In [37]:
next(iter(d.items()))

(10,
 {'loss_CrossEntropyLoss': 0.1305517947420161, 'Accuracy': 0.9615833163261414})

## SweepResult class

In [10]:
sweepdir = '/system/user/beck/pwbeck/projects/regularization/erank/outputs/mnist-11.6.0-conv4_rotatedtasks--221015_124319'
sweepdir = '/system/user/beck/pwbeck/projects/regularization/erank/outputs/mnist-11.7.0-lenet_rotatedtasks--221015_124528'

In [11]:
sweepr = SweepResult(sweepdir)
sweepr.get_sweep_params()

['trainer.init_model_step', 'data.dataset_kwargs.rotation_angle']

In [12]:
dfs = sweepr.get_summary(searchstr='step-0')
dfs

100%|██████████| 360/360 [00:11<00:00, 31.34it/s]


,best_step,best_val_score,trainer.init_model_step,data.dataset_kwargs.rotation_angle,seed
mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-0.0-seed-0--221015_195056,1550,0.976750,0,0.000000,0
mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-0.5013927576601671-seed-0--221015_135150,1650,0.977750,0,0.501393,0
mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-1.0027855153203342-seed-0--221015_184006,1550,0.977167,0,1.002786,0
mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-1.5041782729805013-seed-0--221015_195009,1550,0.976917,0,1.504178,0
mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-10.027855153203342-seed-0--221015_171532,1750,0.976750,0,10.027855,0
...,...,...,...,...,...
mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-97.77158774373258-seed-0--221015_151324,1550,0.977250,0,97.771588,0
mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-98.27298050139275-seed-0--221015_174344,1850,0.977250,0,98.272981,0
mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-98.77437325905292-seed-0--221015_195024,1950,0.979000,0,98.774373,0
mnist-11.7.0-lenet_rotatedtasks--init_model_step-0-rotation_angle-99.27576601671309-seed-0--221015_202015,1850,0.977583,0,99.275766,0


In [13]:
bool(searchstr)

NameError: name 'searchstr' is not defined